In [1]:
# plaidml

import plaidml.keras
plaidml.keras.install_backend()

In [2]:
# Data Loader

import csv
import numpy as np
from PIL import Image
from keras.preprocessing import image
from keras.utils import Sequence, to_categorical

def load_csv(folder, csvname):
    # Read CSV Data
    _file = open('{}/{}'.format(folder, csvname), 'r', encoding='utf-8')
    _reader = csv.reader(_file)
    
    filenames = []
    labels = {}
    for line in _reader:
        _x = '{}/{}'.format(folder, line[0])
        _y = [int(it) for it in line[1:]]
        filenames.append(_x)
        labels[_x] = _y
    _file.close()
    
    return filenames, labels

class SVHNLoader(Sequence):
    def __init__(self, filenames, labels, batch_size=32, dim=(48, 48), n_channels=1,
                 shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.filenames = filenames
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.filenames) / self.batch_size))
    
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        filenames_temp = [self.filenames[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(filenames_temp)

        return X, y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, filenames_temp):
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, 2))

        # Generate data
        for idx, filename in enumerate(filenames_temp):
            # Store sample
            X[idx,] = self.__preprocess_image(filename)

            # Store class
            _y = self.labels[filename]
            y[idx] = to_categorical(_y)

        return X, y
    
    def __preprocess_image(self, filename):
        '''
        Image Preprocessing (grayscale)
        '''
        img = image.load_img(filename, target_size=(48, 48))
        img_tensor = image.img_to_array(img)
        img_tensor = np.dot(img_tensor[...,:3], [0.299, 0.587, 0.114])
        img_tensor = np.reshape(img_tensor, (48, 48, 1))
        img_tensor /= 255.0
        img_tensor = img_tensor - img_tensor.mean()
        return img_tensor

In [3]:
filenames = {'train': None, 'test': None}
labels = {'train': None, 'test': None}
filenames['train'], labels['train'] = load_csv('./datasets/checker_train', 'labels.csv')
filenames['test'], labels['test'] = load_csv('./datasets/checker_test', 'labels.csv')

train_dl = SVHNLoader(filenames['train'], labels['train'], batch_size=128)
test_dl = SVHNLoader(filenames['test'], labels['test'], batch_size=128)

In [4]:
# Keras

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Input, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Flatten
from keras import optimizers
from keras import backend as K
from tensorflow.python.framework import graph_util, graph_io

# Set the float64 as model saver dtype, it will help while serving in golang
# print("Current floating format for Keras backend :", K.floatx())
# floatx = "float64"
# keras.backend.set_floatx(floatx)
# print("Current floating format for Keras backend :", K.floatx())

In [15]:
#from keras.applications import VGG16
from keras.metrics import categorical_accuracy

def designed_model_cnn():
    inp = Input(shape=(48, 48, 1), name='input_layer')
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(inp)
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=2)(x)
    
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(2, activation='softmax', name='output_layer')(x)
    
    model = keras.Model(inputs=inp, outputs=out)
    optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

def designed_model():
    inp = Input(shape=(48, 48, 1), name='input_layer')
    x = Flatten(name='flatten_layer')(inp)
    x = Dense(512, activation='relu', name='layer1')(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu', name='layer2')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu', name='layer3')(x)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu', name='layer4')(x)
    x = BatchNormalization()(x)
    out = Dense(2, activation='softmax', name='output_layer')(x)
    
    model = keras.Model(inputs=inp, outputs=out)
    optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=True)
    model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model

# designed_model().summary()
#designed_model_cnn().summary()

In [16]:
sess = K.get_session()
model = designed_model_cnn()
# model = designed_model()
model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss')
]

# model1.fit_generator(generator=train_dl,
#                     validation_data=test_dl,
#                     use_multiprocessing=True,
#                     workers=6,
#                     epochs=25,
#                     callbacks=callbacks,
#                     verbose=0)


model.fit_generator(generator=train_dl,
                    validation_data=test_dl,
                    epochs=25,
                    callbacks=callbacks)

# model.fit_generator(generator=train_dl,
#                     validation_data=test_dl,
#                     epochs=25)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 48, 48, 16)        160       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 48, 48, 16)        2320      
_________________________________________________________________
batch_normalization_11 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 24, 24, 32)        4640      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 24, 24, 32)        9248

ValueError: Error when checking target: expected output_layer to have shape (1,) but got array with shape (2,)

In [8]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='./model_plot.png', show_shapes=True, show_layer_names=True)

In [6]:
# Save model as protobuf format
constant_graph = graph_util.convert_variables_to_constants(sess,
                                                          sess.graph.as_graph_def(),
                                                          [model.output.op.name])

MODEL_SAVER_DIR_NAME = "checker_model_0824"
MODEL_FILE_NAME = "checker_model" + ".pb"
graph_io.write_graph(constant_graph, MODEL_SAVER_DIR_NAME, MODEL_FILE_NAME, as_text=False)

# close the session
# sess.close()

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 26 variables.
INFO:tensorflow:Converted 26 variables to const ops.


'checker_model_0824/checker_model.pb'

In [7]:
import time

def read_img(filename):
    img = image.load_img(filename, target_size=(48, 48))
    img_tensor = image.img_to_array(img)
    img_tensor = np.dot(img_tensor[...,:3], [0.299, 0.587, 0.114])
    img_tensor = np.reshape(img_tensor, (48, 48, 1))
    img_tensor /= 255.0
    img_tensor = img_tensor - img_tensor.mean()
    return img_tensor


def predict(filename):
    img = read_img(filename)
    dat = model.predict(np.array([img]))
    return dat

def predict_with_time(folder, filename):
    t1 = time.time()
    output = predict('{}/{}'.format(folder, filename))
    t2 = time.time()
    
    print('=' * 12, filename, '=' * 12)
    print('Raw Output :', output)
    print('Containing Number :', 'N' if output[0][0] > output[0][1] else 'Y')
    print('Elapsed Time :', round(t2 - t1, 6))
    return output
    

folder = './datasets/checker_train'
out = predict_with_time(folder, 'f12224.png')

ValueError: Error when checking input: expected input_layer to have 3 dimensions, but got array with shape (1, 48, 48, 1)